In [2]:
import os

In [3]:
%pwd

'c:\\Users\\User\\Documents\\EndToEndMLProjects\\End-To-End-Machine-Learning-Project-with-MlFlow\\research'

In [4]:
# Go back 1 folder to be in the project folder
os.chdir("../")
%pwd

'c:\\Users\\User\\Documents\\EndToEndMLProjects\\End-To-End-Machine-Learning-Project-with-MlFlow'

4. Update the entity

- Copy the import and `DataIngestionConfig class` and paste it in src/entity/config_entity_.py

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

5. Update the configuration manager in src config

- Copy the import and `ConfigurationManager class` and paste it in src/config/configuration.py

In [ ]:
# Read yaml file from 'constants/__init__.py' file

from mlproject.constants import * 
from mlproject.utils.common import read_yaml, create_directories

In [ ]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH, # file path from 'constants' folder 
        params_filepath = PARAMS_FILE_PATH, # file path from 'constants' folder
        schema_filepath = SCHEMA_FILE_PATH # file path from 'constants' folder
        ):
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        # Creates 'artifacts' folder
        create_directories([self.config.artifacts_root])
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        """Function to return data_ingestion directories.

        Returns:
            DataIngestionConfig: DataIngestionConfig type.
        """
        # Access all 'data_ingestion' variables from config.yaml
        config = self.config.data_ingestion
        
        # Creates data_ingestion directory
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        
        return data_ingestion_config

6. Update the components

- Create a file if not exist in src/components named `data_ingestion.py`
- Copy the `import` and `DataIngestion class` and paste it in src/components/data_ingestion.py

In [ ]:
import os
import urllib.request as request
import zipfile
from pathlib import Path
from mlproject import logger
from mlproject.utils.common import get_size

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
    def download_file(self):
        """Download the .zip file.
        """
        # Ensure the file to download is a .zip
        if not self.config.source_URL.endswith(".zip"):
            raise ValueError("Check data_ingestion config.yaml file, source_URL is not a .zip file.")
        
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url=self.config.source_URL,
                filename=self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")
            
    def extract_zip_file(self):
        """Extracts the zip file into the directory.
        """
        unzip_path = self.config.unzip_dir        
        
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(file=self.config.local_data_file, mode='r') as zip_ref:
            zip_ref.extractall(unzip_path)
        

7. Update the pipeline 

- Create a file if not exists in src/pipeline named `stage_01_data_ingestion.py`. First Step in pipeline.
- import necessary class: `ConfigurationManager`, `DataIngestion`, and `logger`
- Copy the code below and paste it in `src/pipeline/stage_01_data_ingestion.py`

In [13]:
# Test run
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
    
except Exception as e:
    raise e

[2024-11-15 10:52:17,433: 35 mlprojectLogger: INFO: common: .yaml file: config\config.yaml loaded successfully.]
[2024-11-15 10:52:17,446: 35 mlprojectLogger: INFO: common: .yaml file: params.yaml loaded successfully.]
[2024-11-15 10:52:17,446: 35 mlprojectLogger: INFO: common: .yaml file: schema.yaml loaded successfully.]
[2024-11-15 10:52:17,487: 54 mlprojectLogger: INFO: common: Created directory at artifacts]
[2024-11-15 10:52:17,489: 54 mlprojectLogger: INFO: common: Created directory at artifacts/data_ingestion]
[2024-11-15 10:52:22,294: 17 mlprojectLogger: INFO: 3003930919: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 5123932
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "8cc8e409e45ac3f6e17b1e8a257b6c9549f254d81c4b9475fa6d620e514025b3"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: d

8. Update the main.py

- To test pipeline add it to `main.py`